# Connect Besu

In [16]:
import websocket

In [ ]:
import json, re
from web3 import Web3
from web3.middleware import geth_poa_middleware
import openstack
from dotenv import dotenv_values
env = dotenv_values("../caliper-benchmarks/cc.env")

def create_connection(auth_url, region, project_name, username, password,
                      user_domain, project_domain, project_id):
    return openstack.connect(
        auth_url=auth_url,
        project_name=project_name,
        username=username,
        password=password,
        region_name=region,
        user_domain_name=user_domain,
        project_domain_name=project_domain,
        project_id=project_id,
        app_name='bpet',
        app_version='1.0',
    )

def collect_info():
    httpUrls, wsUrls = {}, {}
    nodes, ipAddresses = [], []
    conn = create_connection(auth_url=env['OS_AUTH_URL'], region=env['OS_REGION_NAME'],
        project_name=env['OS_PROJECT_NAME'], username=env['OS_USERNAME'],
        password=env['OS_PASSWORD'], user_domain=env['OS_USER_DOMAIN_NAME'],
        project_domain=env['OS_PROJECT_DOMAIN_NAME'], project_id=env['OS_PROJECT_ID'])
    for server in conn.compute.servers():
        if(re.match(r'besu-\d+', server.name)):
            node = server.name
            nodes.append(node)
            instance_id = conn.compute.find_server(node).id
            host_ip = conn.compute.get_server(instance_id).addresses['rrg-khazaei-network'][0]['addr']
            httpUrls[node] = 'http://{}:8545'.format(host_ip)
            wsUrls[node] = 'ws://{}:8546'.format(host_ip)
            ipAddresses.append(host_ip)
    return httpUrls, wsUrls, ipAddresses, nodes

httpUrls, wsUrls, ipAddresses, nodes = collect_info()
print(httpUrls)
print(wsUrls)

In [ ]:
w3 = Web3(Web3.HTTPProvider(httpUrls[nodes[0]]))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.isConnected()
len(w3.geth.admin.peers())

In [ ]:
urls = httpUrls
for k,v in urls.items():
    provider = Web3.HTTPProvider(v)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    peers = w3.geth.admin.peers()
    print(k, 'has', len(peers), 'peers')

In [ ]:
# send an HTTP POST request to node 0
import requests
def call(data, url):
    response = requests.post(url, data)
    return response.json()['result']
data = '{"jsonrpc":"2.0","method":"ibft_getSignerMetrics","params":["1", "1000"], "id":1}'
res = call(data, urls[nodes[0]])
res

## Websocket request

In [7]:
ws = websocket.WebSocket()
ws.connect("ws://192.168.226.109:8546")
ws.send('{"jsonrpc":"2.0","method":"ibft_getSignerMetrics","params":["1", "1000"], "id":1}')
print(ws.recv())

{"jsonrpc":"2.0","id":1,"result":[{"address":"0xd3100a2745437deac5e404d6bb4e254e1f897e49","proposedBlockCount":"0x3e","lastProposedBlockNumber":"0x3da"},{"address":"0xd3e3869a3527c0a0b682a557665a3d3207adc4c8","proposedBlockCount":"0x3e","lastProposedBlockNumber":"0x3db"},{"address":"0x6ae48a0942dd1250f94f7a23fce5212e11d1fce8","proposedBlockCount":"0x3f","lastProposedBlockNumber":"0x3e3"},{"address":"0xb0c57f2fafc548c14cdb9ded00d4b5f504ce6ac2","proposedBlockCount":"0x3e","lastProposedBlockNumber":"0x3d9"},{"address":"0xe0b6beafa380327f02a2a14c0c73d3499e19d818","proposedBlockCount":"0x3e","lastProposedBlockNumber":"0x3dc"},{"address":"0x914ff403d86c9afdb323c09485893f3530fa22b1","proposedBlockCount":"0x3f","lastProposedBlockNumber":"0x3e6"},{"address":"0x7a400f14cecf98a6914e522644b46f56f816b60e","proposedBlockCount":"0x3f","lastProposedBlockNumber":"0x3e5"},{"address":"0x0e1fad5411acef71d91120b74b4ad897dddb5a2e","proposedBlockCount":"0x3e","lastProposedBlockNumber":"0x3df"},{"address":"0x

In [29]:
ws = websocket.WebSocket()
ws.connect("ws://192.168.226.10:8546")
ws.send('{"jsonrpc":"2.0","method":"ibft_getSignerMetrics","params":["1", "1000"], "id":1}')
print(ws.recv())

{"jsonrpc":"2.0","id":1,"result":[{"address":"0x5f2f71a4081e14f1c148abd5ae0bad56a8f95e12","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x173"},{"address":"0x85c22b8213052b1a35fb7f53c12c6571641fe6bd","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x175"},{"address":"0x8966f9688eccdecad4e1b8abe6345f47db5863ec","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x176"},{"address":"0x62e263c4aece42f7d8cacb6c4630125adefb9f0c","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x174"},{"address":"0x4b72c8817621bbafb0c13fb86fe325df04dff44d","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x172"},{"address":"0xf4a6d02060ee344bb26dcc98633ff2399c8d9d9e","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x177"},{"address":"0x2aa94c4502d98e426d36ffb7fde2f24821def1ca","proposedBlockCount":"0x30","lastProposedBlockNumber":"0x179"},{"address":"0x13bc976505f60f7252d5770b71d15c1b88db30ce","proposedBlockCount":"0x2f","lastProposedBlockNumber":"0x178"}]}


In [21]:
ws = websocket.WebSocket()
ws.connect("ws://10.2.12.109:8546")
ws.send('{"jsonrpc":"2.0","method":"ibft_getSignerMetrics","params":["1", "1000"], "id":1}')
print(ws.recv())

OSError: [Errno 101] Network is unreachable

# Compile Smart Contracts

In [ ]:
from solcx import install_solc
install_solc(version='latest')

In [ ]:
from solcx import compile_source
import json

compiled_contract = '../build/contracts/SimpleStorage.json'
with open(compiled_contract) as f:
    data = json.load(f)
    abi = data['abi']
    bin = data['bytecode']

compiled_sol = compile_source(
    '''
    // SPDX-License-Identifier: MIT
    pragma solidity >=0.7.0;

    contract SimpleStorage {
        uint public storedData;

        constructor(uint initVal) {
            storedData = initVal;
        }

        function set(uint x) public {
            storedData = x;
        }

        function get() view public returns (uint retVal) {
            return storedData;
        }
    }
    ''',
    output_values=['abi', 'bin']
)
contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

# Ganache

In [ ]:
w3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))
simpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
w3.eth.default_account = w3.eth.accounts[0]

# submit the transaction that deploys the contract
tx_hash = simpleStorage.constructor(100).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

In [ ]:
s = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
print(w3.eth.get_code(s.address))
s.functions.get().call()

In [ ]:
tx_hash = s.functions.set(200).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
s.functions.get().call()

In [ ]:
data = '{"jsonrpc":"2.0","method":"admin_nodeInfo","params":[],"id":1}'
nodeinfo = call(data, urls[nodes[0]])
nodeinfo

In [ ]:
# adding a validator
data = '{"jsonrpc":"2.0","method":"ibft_proposeValidatorVote","params":["0x4fe188ada84c5611c4cf685e1fd98336b9b5cc9d", true], "id":1}'
for node in nodes[:4]:
    url = urls[node]
    response = requests.post(url=url, data=data)
    print(node, ': ', response.json())

In [ ]:
data = '{"jsonrpc":"2.0","method":"ibft_getValidatorsByBlockNumber","params":["latest"], "id":1}'
response = requests.post(urls[nodes[0]], data=data)
response.json()

In [ ]:
# removing a validator
data = '{"jsonrpc":"2.0","method":"ibft_proposeValidatorVote","params":["0x4fe188ada84c5611c4cf685e1fd98336b9b5cc9d", false], "id":1}'
for node in nodes[:4]:
    url = urls[node]
    response = requests.post(url=url, data=data)
    print(node, ': ', response.json())

In [ ]:
data = '{"jsonrpc":"2.0","method":"ibft_getValidatorsByBlockNumber","params":["latest"], "id":1}'
validators = call(url=urls[nodes[0]], data=data)
validators

In [ ]:
data = '{"jsonrpc":"2.0","method":"net_enode","params":[],"id":1}'
enode = call(url=urls[nodes[0]], data=data)
enode

In [ ]:
# return chainID
data = '{"jsonrpc":"2.0","method":"eth_chainId","params":[],"id":51}'
chainId = call(url=urls[nodes[0]], data=data)
int(chainId, 0)

In [ ]:
# Returns the index corresponding to the block number of the current chain head.
import datetime
data = '{"jsonrpc":"2.0","method":"eth_blockNumber","params":[],"id":51}'
blockNum = call(url=urls[nodes[0]], data=data)
print(datetime.datetime.now(), int(blockNum, 0))

In [ ]:
data = '{"jsonrpc":"2.0","method":"eth_protocolVersion","params":[],"id":1}'
protocalVersion = call(data, urls[nodes[0]])
int(protocalVersion, 0)

In [ ]:
# txHash = '0xc22f96855f20417ad9bf156bddc1a0244be947294de5aeaa6ae287a1771f409f'
data = '{"jsonrpc":"2.0","method":"eth_getTransactionReceipt","params":["0xc22f96855f20417ad9bf156bddc1a0244be947294de5aeaa6ae287a1771f409f"],"id":53}'
receipt = call(data, urls[nodes[0]])
receipt

In [ ]:
# get block by number
from datetime import datetime
data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["latest", true],"id":1}'
block = call(data)
startBlock = int(block['number'], 0)
print(datetime.now(), startBlock)

In [ ]:
data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["latest", true],"id":1}'
block = call(data)
endBlock = int(block['number'], 0)
print(datetime.now(), endBlock)

In [ ]:
blocks = []
for i in range(startBlock, endBlock):
    data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["%d", true],"id":1}' % i
    block = call(data)
    if len(block['transactions']) != 0:
        print(int(block['number'], 0), 'has', len(block['transactions']), 'tranactions')
        blocks.append(int(block['number'], 0))

In [ ]:
for block in blocks:
    data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["%d", true],"id":1}' % block
    blockData = call(data)
    value = int(blockData['transactions'][0]['value'], 0)
    if value != 0:
        print(block, 'has value ', value)

In [ ]:
data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["%d", true],"id":1}' % 63615
block = call(data)
block

In [ ]:
data = '{"jsonrpc":"2.0","method":"eth_getBlockByNumber","params":["0x6a3e", true],"id":1}'
block = call(data)
block

In [ ]:
int('0xde0b6b3a7640000', 0)

In [ ]:
w3.eth.get_block(blocks[10]).size

In [ ]:
w3.eth.get_block(blocks[10]).size

In [ ]:
# block size comparison: 63615 (200tx) vs 0(0tx)
blks = [0, 63615]
for b in blks:
    print('block ', b, ' size is ', w3.eth.get_block(b).size, ' Bytes')

# Rinkeby Test

In [ ]:
import requests

# rrg Rinkeby Besu node: eth-rinkeby-cx
ipAddresses = ['206.12.93.74']
# get a list of peers/ip addresses of eth-rinkeby-cx
data = '{"jsonrpc":"2.0","method":"admin_peers","params":[],"id":1}'
response = requests.post(url='http://'+ipAddresses[0]+':8545', data=data)
for res in response.json()['result']:
    ipAddresses.append(res['enode'].split('@')[1].split(':')[0])
urls = ['http://'+ip+':8545' for ip in ipAddresses]
# trying to connect each peer
for url in urls:
    provider = Web3.HTTPProvider(url)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    # if connected successfully, send a request to add eth-rinkeby-cx as a signer
    if(w3.isConnected()):
        data = '{"jsonrpc":"2.0","method":"clique_propose","params":["0x4fe188ada84c5611c4cf685e1fd98336b9b5cc9d", true], "id":1}'
        response = requests.post(url=url, data=data)
        print(url, ' added signer ', response)

In [ ]:
data = '{"jsonrpc":"2.0","method":"clique_getSigners","params":["latest"], "id":1}'
response = requests.post(url=urls[0], data=data)
response

In [ ]:
response.json()

# Besu Eth Transactions

In [ ]:
import subprocess, os, json
from web3 import Web3

w3 = Web3(Web3.HTTPProvider(httpUrls[nodes[0]]))
print('Connected to Besu: ', w3.isConnected())
genesisFile = "genesis.json"
# key = "../data/rrg-bpet"
# subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
#         "ubuntu@{}:/home/ubuntu/bpet/deploy/{}".format(ipAddresses[0], genesisFile), os.getcwd()], stdout=None)
with open(genesisFile, 'r') as f:
    data = json.load(f)['alloc']

addrKeys = [addr for addr in data.keys()]
addrs = [Web3.toChecksumAddress(addr) for addr in addrKeys]
print('The first privateKey:', data[addrKeys[0]]['privateKey'])
def get_balance(address):
    balance_bank = w3.eth.getBalance(address)
    return w3.fromWei(balance_bank, "ether")
for i in range(len(addrs)):
    addr = addrs[i]
    addrKey = addrKeys[i]
    print(addr, ': ', get_balance(addr), data[addrKey]['privateKey'])

In [ ]:
# send transactions: send value from one account to another (stephen's account)
to_addr = '0x2c45CbE25efD7442AAbbAF6633A88Ef4E139586d'
from_addr = addrs[9]
print('Origianal balance of ',to_addr, ': ', get_balance(to_addr))
txn_opts = {
    'from': addrs[9],
    'to': to_addr,
    'value': w3.toWei(100, 'ether'),
    'gas': 90000,
    'gasPrice': 18000000000,
    'nonce': w3.eth.getTransactionCount(from_addr),
    'chainId': 1337
}
from_priv_key = data[addrKeys[9]]['privateKey']
signed_txn = w3.eth.account.sign_transaction(txn_opts, from_priv_key)
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print('New balance of ',to_addr, ': ', get_balance(to_addr))
receipt

# Besu Smart Contracts

In [ ]:
import subprocess, os, json
from web3 import Web3

w3 = Web3(Web3.HTTPProvider(httpUrls[nodes[0]]))
print('Connected to Besu: ', w3.isConnected())
genesisFile = "genesis.json"
# key = "../data/rrg-bpet"
# subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
#         "ubuntu@{}:/home/ubuntu/bpet/deploy/{}".format(ipAddresses[0], genesisFile), os.getcwd()], stdout=None)
with open(genesisFile, 'r') as f:
    data = json.load(f)['alloc']

addrKeys = [addr for addr in data.keys()]
addrs = [Web3.toChecksumAddress(addr) for addr in addrKeys]
print('The first privateKey:', data[addrKeys[0]]['privateKey'])
def get_balance(address):
    balance_bank = w3.eth.getBalance(address)
    return w3.fromWei(balance_bank, "ether")
for i in range(len(addrs)):
    addr = addrs[i]
    addrKey = addrKeys[i]
    print(addr, ': ', get_balance(addr), data[addrKey]['privateKey'])

In [ ]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
w3 = Web3(Web3.HTTPProvider(httpUrls[nodes[3]]))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

genesisFile = 'genesis.json'
with open(genesisFile, 'r') as f:
    data = json.load(f)['alloc']
addrKeys = [addr for addr in data.keys()]
addrs = [Web3.toChecksumAddress(addr) for addr in addrKeys]

print('Connected to Besu: ', w3.isConnected(), 'Node: ', httpUrls[nodes[3]], w3.clientVersion)

with open('../build/contracts/SimpleStorage.json') as file:
    compiled = json.load(file)
contractAbi = compiled['abi']
contractBin = compiled['bytecode']
contractInit = "000000000000000000000000000000000000000000000000000000000000002F"
fromAddress = addrs[3]

rawtxOptions = {
    'nonce': w3.eth.get_transaction_count(fromAddress),
    'from': fromAddress,
    'to': None,
    'value': '0x00',
    'data': contractBin+contractInit,
    'gas': 90000,
    'gasPrice': 18000000000,
    'chainId': 1337
}

priv_key = data[addrKeys[3]]['privateKey']
signed_txn = w3.eth.account.sign_transaction(rawtxOptions, priv_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
receipt

In [ ]:
w3.eth.get_code(receipt.contractAddress)

In [ ]:
contract = w3.eth.contract(abi=contractAbi)

In [ ]:
contractAddress = '0x674e099fACd86e7267dF794F72ea38274bbA86A0'
contract = w3.eth.contract(address=contractAddress, abi=contractAbi)
w3.eth.getCode(contract.address)

contract.functions.get().call()

In [ ]:
contracts = ['EnergyCoin', 'PaymentSettlement', 'PoolMarket']
contractAddresses = {'EnergyCoin': '0x8eB9147a7b1cfD5d20dC5bc15ca60c16F0D871cb',
    'PaymentSettlement': '0xf1D795F56996eC57820449Ee14Dd710dAb0c8E8f',
    'PoolMarket': '0x122Aab2C3A3f73200315b2e417236259415c7C25'}
contracts = contractAddresses.keys()

abis = {}
for contract in list(contracts):
    with open('../build/contracts/{}.json'.format(contract)) as file:
        compiled = json.load(file)
    contractAbi = compiled['abi']
    abis[contract] = contractAbi

In [ ]:
EnergyCoin = w3.eth.contract(
    address=contractAddresses['EnergyCoin'],
    abi=abis['EnergyCoin'],
)

EnergyCoin.functions.getMyOwnBalance().call()